In [7]:
import pandas as pd
#빌라주택, 원룸투룸  데이터 가져오기 
data = pd.read_csv('../preprocessing_data/최종/living_alone.csv')
data.head()

,x,y
0,126.896310,37.521396
1,126.891338,37.526169
2,126.901841,37.524675
3,126.901250,37.531040
4,126.905976,37.531724
...,...,...
93,126.884470,37.511326
94,126.884052,37.511529
95,127.042302,37.572899
96,127.042946,37.573948


In [9]:
from selenium import webdriver

driver = webdriver.Chrome('../chromedriver.exe')
driver.get('https://new.land.naver.com/houses?ms=37.5214322,126.8973078,19&a=VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL')


In [79]:
from bs4 import BeautifulSoup as bs

page = driver.page_source
soup = bs(page, 'html.parser')
circle = soup.find('div',class_ = 'map_area').select('div > div > a')
for i in circle:
    try:
        print(i['id'])
        print(i.select_one('span.sale_number').text.split('개')[0])
        #여기서 이제 데이터 가져오기 
    except:
        break

21203331102LGEOHASH_MIX_ARTICLE
11
21203331103LGEOHASH_MIX_ARTICLE
46
21203331112LGEOHASH_MIX_ARTICLE
7
21203331113LGEOHASH_MIX_ARTICLE
9
21203331120LGEOHASH_MIX_ARTICLE
16
21203331121LGEOHASH_MIX_ARTICLE
9
21203331122LGEOHASH_MIX_ARTICLE
4
21203331123LGEOHASH_MIX_ARTICLE
3
21203331131LGEOHASH_MIX_ARTICLE
1
21203331133LGEOHASH_MIX_ARTICLE
3
21203331311LGEOHASH_MIX_ARTICLE
7
21203331313LGEOHASH_MIX_ARTICLE
6
21212220002LGEOHASH_MIX_ARTICLE
6
21212220003LGEOHASH_MIX_ARTICLE
8
21212220012LGEOHASH_MIX_ARTICLE
1
21212220013LGEOHASH_MIX_ARTICLE
44
21212220020LGEOHASH_MIX_ARTICLE
14
21212220021LGEOHASH_MIX_ARTICLE
25
21212220022LGEOHASH_MIX_ARTICLE
5
21212220023LGEOHASH_MIX_ARTICLE
15
21212220030LGEOHASH_MIX_ARTICLE
25
21212220031LGEOHASH_MIX_ARTICLE
18
21212220032LGEOHASH_MIX_ARTICLE
27
21212220033LGEOHASH_MIX_ARTICLE
18
21212220200LGEOHASH_MIX_ARTICLE
2
21212220201LGEOHASH_MIX_ARTICLE
16
21212220202LGEOHASH_MIX_ARTICLE
24
21212220203LGEOHASH_MIX_ARTICLE
9
21212220210LGEOHASH_MIX_ARTICLE
25


In [130]:
from selenium.webdriver.common.keys import Keys
import time
from collections import defaultdict

def circle_data_crawling(circle_id, circle_num):
    lst = []
    driver.find_element_by_xpath(f'//*[@id="{circle_id}"]').click()
    #밑으로 내려주기 내려주는거 횟수 저장하기 
    for _ in range(circle_num // 20 + 1):
        itemlist = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
        driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", itemlist)
        time.sleep(0.5)
        
    for i in range(1, circle_num + 1):
        data = defaultdict(str)
        #제목누르기 사진이 있는거와 없는거가 찾이가 난다 //*[@id="listContents1"]/div/div/div/div[1]/div/a[2]/div[1]/span
        try:
            driver.find_element_by_xpath(f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a/div[1]/span').click()
            #사진 없음
            data['사진'] = 0
        except:
            driver.find_element_by_xpath(f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a[2]/div[1]/span').click()
            data['사진'] = 1
        time.sleep(0.5)
        estimate = driver.page_source
        estimate_soup = bs(estimate, 'html.parser')
        detail_estimate = estimate_soup.find('div', class_ = 'detail_contents_inner')

        data['제목'] = detail_estimate.find('h4').text # 제목
        data['가격'] = detail_estimate.find('div', class_ ='info_article_price' ).text # 가격
        data['소제목'] = detail_estimate.find('div', class_ ='info_article_feature' ).text #소제목

        # 표 데이터 
        for i in detail_estimate.find('div', class_ = 'detail_box--summary').find_all('tr'):
            for title, content  in zip(i.find_all('th'), i.find_all('td')):
                data[title.text] = content.text
        #데이터 저장
        lst.append(data)

    return lst


[print(i) for i in circle_data_crawling('21231000121303LGEOHASH_MIX_ARTICLE',6)]

defaultdict(<class 'str'>, {'사진': 0, '제목': '일반원룸2층', '가격': '월세500/50', '소제목': '해당층/총층: 2/4층오픈형/단층 전용 면적:23㎡', '소재지': '경기도 양평군 옥천면 아신리 435-3', '매물특징': '남한강 자전거길 따뜻한 남향 원룸', '공급/전용면적': '118.08㎡/23㎡(전용률19%)', '해당층/총층': '2/4층', '방수/욕실수': '1/1개', '월관리비': '5만원', '관리비 포함': '수도, 인터넷, TV', '입주가능일': '즉시입주 ', '융자금': '없음', '사용승인일': '2017.03.21', '방향': '남향(거실 기준)', '주차가능여부': '가능', '방구조': '오픈형', '복층여부': '단층', '건축물 용도': '단독주택', '매물번호': '2243851159', '총주차대수': '13대', '매물설명': '남한강 자전거길 바로 앞 따뜻한 원룸\r\n\r\n따뜻한 남향의 햇살가득한 원룸입니다.\r\n풀옵션, 마트 및 편의시설 도로 1분거리\r\n아신역 도보 5분거리로 교통 입지가 좋습니다. \r\n\r\n세입자분께서 워낙 깔끔하게 쓰셔서 \r\n깔끔한 원룸 찾는분께 추천드립니다. \r\n\r\n월세 문의 010-9220-6886', '중개사': '양평시대 공인중개사사무소길찾기대표김정희등록번호41830202000018소재지경기도 양평군 강상면 강남로 697 101호 (병산리)전화031-775-6993,031-775-6993최근 3개월  집주인확인 31건매매25전세0월세1단기0'})
defaultdict(<class 'str'>, {'사진': 0, '제목': '일반원룸4층', '가격': '전세7,000(771만원/3.3㎡)', '소제목': '해당층/총층: 4/4층오픈형/단층 전용 면적:22㎡', '소재지': '경기도 양평군 옥천면 아신리 435-3', '매물특징': '아신역 인근 강뷰 나오는 풀옵션 원룸', '공급/전용면적': '30㎡/22㎡(전용률73

[None, None, None, None, None, None]